# Imports

In [1]:
!pwd
import sys
from bumps.names import Curve, fit, FitProblem
from bumps.dream.state import load_state
import matplotlib.pyplot as plt
import numpy as np
import scipy.fft as fft
import os

sys.path.append("../../../../src/molgroups")
import molgroups as mol

sys.path.append("../../../../src/scatpy")
import rs
import rsdi

sys.path.append("../../../../src/infotheory")
import entropy

/Users/frank/Dropbox/My Mac (PN115993.campus.nist.gov)/Documents/programming/molgroups/examples/information_theory/entropy_gridsearch_singlethread/garefl


# Create Short Example Fit

In [2]:
%%writefile setup.cc
#include <iostream>
#include <cassert>
#include "setup.h"
#include "stdio.h"
#include "refl.h"
#include "reflcalc.h"
#include "molgroups.cc"

#define FWHM 2.354820045   // ga_refl uses FWHM. Divide by FWHM to get sigma units.

//reflectivity
#define MODELS 1

//canvas for continuous distribution model
#define CANVASSTART 1
#define DIMENSION 200
#define STEPSIZE 0.5

/* initialising non-standard fitting variables */
double aArea[DIMENSION], anSL[DIMENSION];
double background[MODELS];
char str2[2];

double normarea;
double l_lipid1, l_lipid2, vf_bilayer,  global_rough, rho_siox, l_siox, l_submembrane, sigma;

ssBLM_POPC  bilayer;

void fnSetContrastBilayer(double sigma, double global_rough, double rho_substrate, double rho_siox, double l_siox, double l_submembrane, double l_lipid1, double l_lipid2, double vf_bilayer, fitinfo *fit, int contraststart, int contrastend){
    
    double dMaxArea;
    int i;
    
    for (i=contraststart; i<contrastend+1; i++) {
        bilayer.fnSet(sigma, global_rough, rho_substrate, rho_siox, l_siox, l_submembrane, l_lipid1, l_lipid2, vf_bilayer, 0, 0);
        dMaxArea=fnClearCanvas(aArea, anSL, DIMENSION);
        dMaxArea=bilayer.fnWriteProfile(aArea, anSL, DIMENSION, STEPSIZE, dMaxArea);
        normarea=dMaxArea;
        fnWriteCanvas2Model(aArea,anSL,fit,CANVASSTART,DIMENSION,STEPSIZE,dMaxArea,normarea,i,i);
    }
}

/*=========== CONSTRAINTS =====================*/
void constr_models(fitinfo *fit)
{
    int i;
    //int iPeaked,iMaxPeak, k, i2;
    //fitpars *pars = &fit[0].pars;
    
    
    /* Rescue the free parameters from the model. */
    //for (i=0; i < fit[1].pars.n; i++)
    //    fit[1].pars.value[i] = *(fit[1].pars.address[i]);
    
    /* Go through all layers copying parameters from model 0 to other models */
    //tied_parameters(fit);
    
    
    /* Restore the free parameters to the model. */
    //for (i=0; i < fit[1].pars.n; i++){
    //    *(fit[1].pars.address[i]) = fit[1].pars.value[i];
    //}
    
    for (i=0; i<MODELS; i++) {
        fit[i].beam.background=pow(10,background[i]);
    }
    
    
    //---- d31-POPC bilayer ----
    //----Neat -------
    fnSetContrastBilayer(sigma, global_rough, fit[0].m.rho[0], rho_siox, l_siox, l_submembrane, l_lipid1, l_lipid2, vf_bilayer, fit, 0, 0);
    
    
}

void save(fitinfo *fit)
{
    fnSetContrastBilayer(sigma, global_rough, fit[0].m.rho[0], rho_siox, l_siox, l_submembrane, l_lipid1, l_lipid2, vf_bilayer, fit, 0, 0);
    FILE *fp;
    fp=fopen("mol.dat","w");        
    bilayer.fnWritePar2File(fp,"bilayer",DIMENSION,STEPSIZE);
    fclose(fp);
}


/*============ INITIAL SETUP ============================*/
extern "C"
fitinfo* setup_models(int *models)
{
    static fitinfo fit[MODELS];
    int i,j;
    fitpars *pars = &fit[0].pars;
    fitpars *freepars = &fit[1].pars;
    *models = MODELS;
    
    for (i=0; i < MODELS; i++) fit_init(&fit[i]);
    
    /* Load the data for each model */
    fit_data(&fit[0],"sim0.dat"); /*neat */
    
    /* Initialize instrument parameters for each model.*/
    /* setup for NG7 */
    for (i=0; i < MODELS; i++) {
        
        const double L = 5.00,dLoL=0.015,d=1800.0;
        double Qlo, Tlo, dTlo,dToT,s1,s2;
        Qlo=0.008;
        Tlo=Q2T(L,Qlo);
        s1=0.1, s2=s1;
        dTlo=resolution_dT(s1,s2,d);
        dToT=resolution_dToT(s1,s2,d,Tlo);
        data_resolution_fv(&fit[i].dataA,L,dLoL,Qlo,dTlo,dToT);
        fit[i].beam.lambda = L;
        interface_create(&fit[i].rm, "erf", erf_interface, 21);
    }
    
    /*============= MODEL =====================================*/
    
    /* Add layers: d, rho, mu, rough */
    for (i=0; i < MODELS; i++) {
        model_layer(&fit[i].m, 0.00000, 2.07e-6, 0.0e-8, 3.000);  	/* 0 Si */
        for (j=0; j < DIMENSION; j++) {
            model_layer(&fit[i].m, STEPSIZE, 0.00e-6, 0.0e-8, 0);		/* Canvas */
        }
        model_layer(&fit[i].m, 100.000, 6.35e-6, 0.0e-8, 0.000);		/* Solvent */
    }
    
    /*correct solvent layers for different models*/
    /* fit[3].m.d[3] = ... */

    //headgroup.bWrapping=false;
    
    /*=============== FIT PARAMETERS ===============================*/
    
    /* Specify which parameters are your fit parameters. Parameters are fitted
     * to be the same in all datasets by default
     */
    
    
    pars_add(pars, "l_siox", &(l_siox), 10.0, 30.0);
    pars_add(pars, "rho_siox", &(rho_siox), 3.2e-06, 3.8e-06);
    pars_add(pars, "l_submembrane", &(l_submembrane), 1.0, 10.0);
    pars_add(pars, "l_lipid1", &(l_lipid1), 10.0, 15.0);
    pars_add(pars, "l_lipid2", &(l_lipid2), 10.0, 15.0);
    pars_add(pars, "vf_bilayer", &(vf_bilayer), 0.9, 1.0);
    
    pars_add(pars, "rho_solv_0", &(fit[0].m.rho[fit[0].m.n-1]), -5.399999999999999e-07, 5.999999999999943e-08);
    
    pars_add(pars, "global_rough", &(global_rough), 2.0, 5.0);
    pars_add(pars, "sigma",        &(sigma), 2.0, 5.0);
    
    pars_add(pars, "background_0", &(background[0]), -9.0, -5.0);
    
    /* Build a list of 'free parameters' in fit[1].pars. These are
     * parameters for which the values are aloowed to differ from those
     * in model 0.  By default all values in all models are the same unless 
     * specified here. The range data is not used here, so set it to [0,1].  
     */
    
    //pars_add(freepars, "rho_solv_1", &(fit[1].m.rho[fit[1].m.n-1]), 0, 1);
    
    constraints = constr_models;
    output_model = save;
    return fit;
}


Overwriting setup.cc


In [9]:
!make clean
!make
!cp hk054.refl sim0.dat
!refl1d run.py --fit=dream --store=T --init=lhs --parallel --burn=100 --steps=5 --overwrite

rm -f setup.o
g++  -g -O2 -Wall  -fPIC -DUSE_DOUBLE -DReal=double -DHAVE_CONFIG_H -I/Users/frank/danse/refl1d/garefl/boxmin -I/Users/frank/danse/refl1d/garefl/model1d -I/Users/frank/danse/refl1d/garefl/src -c setup.cc -o setup.o
setup.cc:93:14: warning: unused variable 'freepars' [-Wunused-variable]
    fitpars *freepars = &fit[1].pars;
             ^
1 warning generated.
cd /Users/frank/danse/refl1d/garefl/boxmin && /Applications/Xcode.app/Contents/Developer/usr/bin/make
make[1]: Nothing to be done for `all'.
cd /Users/frank/danse/refl1d/garefl/model1d && /Applications/Xcode.app/Contents/Developer/usr/bin/make
make[1]: Nothing to be done for `all'.
cd /Users/frank/danse/refl1d/garefl/src && /Applications/Xcode.app/Contents/Developer/usr/bin/make
make[1]: Nothing to be done for `all'.
g++    -fPIC -o fit setup.o /Users/frank/danse/refl1d/garefl/src/ga_simul.o /Users/frank/danse/refl1d/garefl/src/ga.o -L/Users/frank/danse/refl1d/garefl/model1d -lrefl -L/Users/frank/danse/refl1d/garefl/b

# Defining Optimization Parameters in simpar.dat

Simpar.dat contains a list of all fit parameters with a designation, whether they are marginal (d) or nuisance (i) parameters. This is followed by the parameter name, the initial parameter value, and the fit boundaries. If three more numbers are given, this designates that an information content search over this parameter is performed (start, stop, step). A preceding f (fi or fd) at the beginning of the line indicates that the fit boundaries for such a search parameter are fixed (for example for volume fractions between 0 and 1), otherwise the fit boundary moves according to the varied parameter and the initally given fit boundaries.

In [1]:
%%writefile entropypar.dat
i l_siox 18 10 30
i rho_siox 3.55e-6 3.20e-6 3.80e-6
i l_submembrane 2.5 1.0 10.0
d l_lipid1 11.8 10.0 15.0
d l_lipid2 12.4 10.0 15.0
d vf_bilayer 0.95 0.90 1.00
i rho_solv_0 6.34e-6 5.8e-6 6.4e-6 -0.5e-6 6.5e-6 0.5e-6
i global_rough 3.0 2.0 5.0
i sigma 2.5 2.0 5.0
i background_0 -8.0 -9.0 -5.0
n prefactor 2.58496

Overwriting entropypar.dat


# Variables

In [4]:
# general fit setup
setupdir = os.getcwd()
runfile = "run"
store = 'T'
fitsource = "garefl"

# particular entropy setup
burn = 8000
steps = 500
convergence = 2.0
miniter = 2
mode = 'water'
bClusterMode = False
bFetchMode = False
time = 2
bcalcsymmetric = True
upper_info_plotlevel = None
plotlimits_filename = ""
calcsingle = False

# setup batchscript for SLURM (if used)
script = []
script.append('#!/bin/bash\n')
script.append('#SBATCH --job-name=entro {mcmc_iteration}\n')
script.append('#SBATCH -A mc4s9np\n')
script.append('#SBATCH -p RM\n')
script.append('#SBATCH -t 0' + str(time) + ':00:00\n')
script.append('#SBATCH -N 4\n')
script.append('#SBATCH --ntasks-per-node 28\n')
script.append('\n')
script.append('set +x\n')
script.append('cd $SLURM_SUBMIT_DIR\n')
# script.append('cd '+dirname+'\n')
script.append('\n')
script.append('module load python/2.7.11_gcc\n')
script.append('export PYTHONPATH=/home/hoogerhe/bin/lib/python2.7/site-packages:/home/hoogerhe/src/bumps\n')
script.append('\n')
script.append('mpirun -np 112 python /home/hoogerhe/src/refl1d/bin/refl1d_cli.py {mcmc_dirname}/run.py --fit=dream --mpi --init=lhs --batch --pop=28 --time=' 
              + str(float(time) - 0.1) + ' --thin=20 --store={mcmc_dirname}/save --burn=' + str(burn) 
              + ' --steps=' + str(steps) + '\n')


# Fit Setup

In [5]:
entr = entropy.Entropy(
    fitsource=fitsource,
    spath=setupdir,
    mcmcpath=store,
    runfile=runfile,
    mcmcburn=burn, 
    mcmcsteps=steps, 
    convergence=convergence, 
    miniter=miniter, 
    mode=mode,                      
    bClusterMode=bClusterMode, 
    bFetchMode=bFetchMode, 
    calc_symmetric=bcalcsymmetric,
    upper_info_plotlevel=upper_info_plotlevel, 
    plotlimits_filename=plotlimits_filename,
    slurmscript=script
)

g++  -g -O2 -Wall  -fPIC -DUSE_DOUBLE -DReal=double -DHAVE_CONFIG_H -I/Users/frank/danse/refl1d/garefl/boxmin -I/Users/frank/danse/refl1d/garefl/model1d -I/Users/frank/danse/refl1d/garefl/src -c setup.cc -o setup.o


setup.cc:93:14: warning: unused variable 'freepars' [-Wunused-variable]
    fitpars *freepars = &fit[1].pars;
             ^
1 warning generated.


cd /Users/frank/danse/refl1d/garefl/boxmin && /Applications/Xcode.app/Contents/Developer/usr/bin/make
make[1]: Nothing to be done for `all'.
cd /Users/frank/danse/refl1d/garefl/model1d && /Applications/Xcode.app/Contents/Developer/usr/bin/make
make[1]: Nothing to be done for `all'.
cd /Users/frank/danse/refl1d/garefl/src && /Applications/Xcode.app/Contents/Developer/usr/bin/make
make[1]: Nothing to be done for `all'.
g++    -fPIC -o fit setup.o /Users/frank/danse/refl1d/garefl/src/ga_simul.o /Users/frank/danse/refl1d/garefl/src/ga.o -L/Users/frank/danse/refl1d/garefl/model1d -lrefl -L/Users/frank/danse/refl1d/garefl/boxmin -lboxmin  -lm 
g++ -shared -o model.so setup.o /Users/frank/danse/refl1d/garefl/src/refl1d.o -L/Users/frank/danse/refl1d/garefl/model1d -lrefl  -lm 


In [6]:
entr.run_optimization()

In [7]:
entr.plot_results()